# What do terrorists really want?
Google Collab [Updated last on 14/4] - https://colab.research.google.com/drive/1599wFpPYiRBZvVURqFiYF0_ivEbWPZZo?usp=sharing

In [4]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import pickle
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import gensim
from gensim.utils import simple_preprocess
import string
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

np.random.seed(2018)

# Visualisation libraries
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Map stuff
# import folium
# from folium.plugins import MarkerCluster

# Misc
# from collections import Counter
# sns.set()
# %matplotlib inline

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Out of all of the fields that's given to us in the dataset, we would like to take a deeper look into 2 relevant text fields: "summary" and "motive".

According to the GTD Codebook, the **"summary"** column gives us an extremely brief narrative summary of the incident which follows the 5W1H (Who,What,When,Where,Why,How) guidelines.

Whereas the **"motive"** field contains an explicit mention in the official reports. There might be general information about the political, social, or economic situation at the time of attack if it was determined by the researchers to have an impact on the motivation of the incident.

Additionally, it is to be noted that these 2 fields were only implemented starting from the year of 1998. Hence, in order to run our data on more valuable data, we will be taking only the events after 1998.

In [6]:
# GTD Dataset from 1970 - 2019
df_all = pd.read_excel("/content/drive/MyDrive/NTU/Sem 1.2/SC1015/SC1015 Mini-Project/globalterrorismdb_0221dist.xlsx")
file_name = 'df_all'
df_all.to_pickle(file_name)  # where to save it, usually as a .pkl
# df.info

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
df = pd.read_pickle(file_name)

# Drop rows that are before the year 1998
df.drop(df[df['iyear'] < 1998].index, inplace=True)

# Drop columns that are not needed
df.drop(df.columns.difference(['summary', 'motive']), 1, inplace=True)

In [9]:
# Creating data into an excel file
df.to_excel('filename1.xlsx', sheet_name = 'New_Sheet')

In [10]:
# Drop rows with NAN for motive
df.dropna(subset = ['motive'], inplace=True)
df = df[df["motive"].str.contains("Unknown")==False]
df = df[df['motive'] != 'The specific motive for the attack is unknown.']
# df.info

Doing some research beforehand, we understand that we would have to use NLP to help us to reach a conclusion to this question. We think that being able to categorise the words being used in the "summary" and "motive" fields would help us to better understand the motives of the terrorists.

The methods that we could try, include:
*   **Topic Modelling**<br>
A science that requires both art and science to identify and quantify the mic of topics within a document. Topic Modelling is used when we have a set of text documents that we would like to find out about the different topics that they cover and then group then by these topics. Two Analysis that we could try here:
  *   Latent Semantic Analysis (LSA)<br>
  This is like Naive Bayes, it is based on the word frequencies of the dataset. The general idea is that the algorithm will count the frequency of the word and group it together with the.
  *   Latent Dirichlet Allocation (LDA)<br>
  This has a fixed set of topics, we define each topiv to be represented by a set of words. LDA simply tries to map all the words to the topics (in some sort of a backward motion).

*   **Topic Classification**<br>
A note that this ML algorithm is supervised. Topic Modelling is unsupervised.
  *   Empirical Topic Classification (ETC)<br>
That combines NLP methods together with human instuition to identity the word features that result in a more meaningful identification of the terrorists' motive categories.

To give some hindsight, some of the perpertrator's motives could include retaliation, intimidation, causing instability or to raise contempt — despise a certain community.

References:<br>
https://www.mdpi.com/2076-0760/11/1/23#<br>
https://monkeylearn.com/topic-analysis/

# Step 1: Dealing with Motive first

This process would involve:
<!-- ![picture](https://drive.google.com/uc?export=view&id=1r-rjElUan2DorRG2gUNEGuKzZg399PTi) -->

References:<br>
https://towardsdatascience.com/text-classification-supervised-unsupervised-learning-approaches-9fd5e01a036<br>
pyLDAvis: https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21


## Data Pre-processing

*   Change the text into lowercase
*   Remove all punctuations
*   Tokenization<br>
Split the text into sentences and the sentences into words.=
*   All stopwords are removed.
*   Words are lemmatized<br>
Where the words written in third person will be changed to first person. Verbs that are in past and future tenses are changed to the present.



In [11]:
# Converting all of the strings into lowercase
df['motive'] = df['motive'].str.lower()

# Removing punctuation
df['motive'] = df['motive'].str.replace('[{}]'.format(string.punctuation), '')

# Removing digits
df['motive'] = df['motive'].str.replace('\d+', '')

df['motive'].head

<bound method NDFrame.head of 67506     it was believed this attack was meant to intim...
67511     it was believed that the attack was a protest ...
67515     it was believed that the attack was a protest ...
67518     this attack was meant to terrorize civilians a...
67527     it was believed that this attack was one of a ...
                                ...                        
201105    the specific motive is unknown however sources...
201118    the specific motive is unknown however sources...
201127    the specific motive is unknown however authori...
201175    the arakan army aa claimed responsibility and ...
201176    the specific motive is unknown however sources...
Name: motive, Length: 23266, dtype: object>

In [12]:
# Removing stopwords
stop = stopwords.words('english')
newStopWords = ['unknown','however','specific','motive','sources','noted','reported','related','victim',\
                'attack','targeted','speculated','claimed','responsibility','incident','stated','scheduled',\
                'larger','part','suspected','carried','accused','may','january','february','march','april','june',\
                'july','august','september','october','november','december','people','believed','meant','considered']
stop.extend(newStopWords)
df['motive'] = df['motive'].apply(lambda x: [item for item in x.split() if item not in stop])
df['motive'].head

<bound method NDFrame.head of 67506     [intimidate, un, inspectors, halt, weapons, in...
67511                [protest, election, results, occurred]
67515                [protest, election, results, occurred]
67518     [terrorize, civilians, government, ahead, cont...
67527     [one, series, retribution, killings, occurred,...
                                ...                        
201105    [trend, violence, activists, since, start, ant...
201118            [boti, lekam, maoists, police, informant]
201127    [authorities, took, place, us, airstrikes, pro...
201175    [arakan, army, aa, kidnappings, conducted, ord...
201176           [godlove, abanda, collaborating, military]
Name: motive, Length: 23266, dtype: object>

In [13]:
# Tokenization — splitting the text into sentences and sentences into words
# df['motive'].apply(word_tokenize)

# df['motive'].head

In [14]:
# Initialise the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize each word - Group together different forms of the same word (e.g Running == Run)
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]
df['motive'] = df['motive'].apply(lemmatize_text)
print(df['motive'].head)

<bound method NDFrame.head of 67506     [intimidate, un, inspector, halt, weapon, insp...
67511                 [protest, election, result, occurred]
67515                 [protest, election, result, occurred]
67518     [terrorize, civilian, government, ahead, conti...
67527     [one, series, retribution, killing, occurred, ...
                                ...                        
201105    [trend, violence, activist, since, start, anti...
201118             [boti, lekam, maoist, police, informant]
201127    [authority, took, place, u, airstrikes, proira...
201175    [arakan, army, aa, kidnapping, conducted, orde...
201176           [godlove, abanda, collaborating, military]
Name: motive, Length: 23266, dtype: object>


In [15]:
# Splitting the Dataset - Wouldn't need to split dataset for this unsupervised learning model
# X_train, X_test = train_test_split(df['motive'], test_size=0.2, random_state=1)

In [16]:
# Creating data into an excel file
df.to_excel('filename.xlsx', sheet_name = 'New_sheet')

## Bag of Words on the Dataset
A popular technique for developing sentiment analysis models is to use a bag-of-words model that transforms documents into vectors where each word in the document is assigned a score.

In [17]:
# Create a dictionary from the df['motive'] containing the number of times a word appears in the training set.
dictionary = gensim.corpora.Dictionary(df['motive'])

# Print words that are in the top 10
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 halt
1 inspection
2 inspector
3 intimidate
4 un
5 weapon
6 election
7 occurred
8 protest
9 result
10 ahead


In [18]:
# For each row, we will create another dictionary to document the number of words and the number of times that each words appear
bagofwords = [dictionary.doc2bow(doc) for doc in df['motive']]

In [19]:
# Printing of the 10th row (Preview)
bagofwords[10]

[(8, 1),
 (13, 1),
 (27, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1)]

## Running LDA using BoW

In [20]:
# Train our LDA model with the gensim models LDAMulticore
# num_topics: the number of requested latent topics to be extracted from the training corpus. We will stick to 6 for now.
lda_bow = gensim.models.LdaMulticore(bagofwords, num_topics=6, id2word=dictionary, workers=3)

for idx, topic in lda_bow.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.016*"victim" + 0.015*"police" + 0.014*"government" + 0.012*"force" + 0.011*"member" + 0.010*"security" + 0.010*"area" + 0.009*"retaliation" + 0.009*"assailant" + 0.008*"group"
Topic: 1 
Words: 0.013*"trend" + 0.013*"shiite" + 0.012*"community" + 0.011*"violence" + 0.011*"sunni" + 0.010*"election" + 0.010*"attack" + 0.009*"day" + 0.009*"sectarian" + 0.009*"government"
Topic: 2 
Words: 0.031*"islamic" + 0.023*"state" + 0.021*"levant" + 0.020*"iraq" + 0.020*"isil" + 0.012*"posited" + 0.011*"retaliation" + 0.010*"force" + 0.009*"government" + 0.009*"alshabaab"
Topic: 3 
Words: 0.025*"election" + 0.012*"islamic" + 0.010*"campaign" + 0.008*"liberation" + 0.008*"intimidate" + 0.007*"day" + 0.007*"violence" + 0.007*"extremist" + 0.007*"government" + 0.006*"attack"
Topic: 4 
Words: 0.027*"violence" + 0.021*"trend" + 0.016*"community" + 0.014*"sectarian" + 0.012*"police" + 0.012*"minority" + 0.012*"state" + 0.012*"military" + 0.012*"shiite" + 0.011*"sunni"
Topic: 5 
Words: 0.0

In [22]:
# Importing pyLDAvis module for visualisation
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models

lda_display = pyLDAvis.gensim_models.prepare(lda_bow, bagofwords, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

     |████████████████████████████████| 1.7 MB 13.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=cc1e7e840aa1b30b89627bbe318994f0863c1bcd18678ef4ebee9bb5dc9ea6fb
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [23]:
# save the model to disk
filename = 'lda_bow.sav'
pickle.dump(lda_bow, open(filename, 'wb'))

## Running LDA using TFIDF
"In information retrieval, tf–idf, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling."<br>

[Taken from Wikipedia](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

In [24]:
# Creating the tf-idf model
from gensim import corpora, models
from pprint import pprint

tfidf = models.TfidfModel(bagofwords)
corpus_tfidf = tfidf[bagofwords]

In [25]:
lda_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=dictionary, workers=3)
for idx, topic in lda_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

Topic: 0 Word: 0.008*"extortion" + 0.008*"demand" + 0.007*"shiite" + 0.006*"islamic" + 0.006*"violence" + 0.006*"sunni" + 0.005*"state" + 0.005*"community" + 0.005*"trend" + 0.005*"iraq"
Topic: 1 Word: 0.012*"levant" + 0.011*"community" + 0.010*"shiite" + 0.010*"iraq" + 0.009*"sunni" + 0.009*"victim" + 0.009*"majority" + 0.009*"minority" + 0.008*"trend" + 0.008*"violence"
Topic: 2 Word: 0.006*"islamic" + 0.006*"election" + 0.006*"security" + 0.005*"state" + 0.005*"area" + 0.005*"government" + 0.005*"iraq" + 0.005*"force" + 0.004*"posited" + 0.004*"victim"
Topic: 3 Word: 0.014*"election" + 0.008*"victim" + 0.008*"taliban" + 0.007*"government" + 0.007*"police" + 0.006*"posited" + 0.006*"upcoming" + 0.006*"ransom" + 0.005*"maoist" + 0.005*"presidential"
Topic: 4 Word: 0.011*"bangladesh" + 0.010*"began" + 0.010*"nationwide" + 0.010*"hartal" + 0.008*"trend" + 0.008*"retaliation" + 0.008*"violence" + 0.008*"election" + 0.006*"alshabaab" + 0.006*"maoist"
Topic: 5 Word: 0.008*"violence" + 0.00

In [26]:
lda_display = pyLDAvis.gensim_models.prepare(lda_tfidf, corpus_tfidf, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [27]:
# save the model to disk
filename = 'lda_tfidf.sav'
pickle.dump(lda_tfidf, open(filename, 'wb'))

BoW and TFIDF are simply based on the frequency of the words found. However, one of the drawbacks of using this method is the understanding of the context. This is where Word Embedding techniques like Word2Vec, Continuous Bag of Words (CBOW), Skipgram would come in.

For the sake of the simplicity of this model, we will not delve into such algorithm but rather just understand that there are many other ways to improve the analysis of the dataset.

## Name the topics obtained using Google [Not working]
It gave me some results before though. I think maybe Google had blocked my IP from runnng the cell too many times. But VPN doesn't work either. So I think they blacklisted my account.

Update, it allowed for me to run again the next day. So I assume that I just simply can't run too many times in a short span of time.

In [28]:
!pip install cssselect

# Thank you to Sam H. who shared this on https://stackoverflow.com/questions/43985683/automatic-labeling-of-lda-generated-topics
from urllib.parse import urlencode, urlparse, parse_qs
from lxml.html import fromstring
from requests import get
from collections import Counter
import re
import requests
from bs4 import BeautifulSoup

def get_srp_text(search_term):
    raw = get(f"https://www.google.com/search?q={search_term}").text
    page = fromstring(raw)

    blob = ""

    for result in page.cssselect("a"):
        for res in result.findall("div"):
            blob += ' '
            blob += res.text if res.text else " "
            blob += ' '
    return blob

def blob_cleaner(blob):
    clean_blob = blob.replace(r'[\/,\(,\),\:,_,-,\-]', ' ')
    return ''.join(e for e in blob if e.isalnum() or e.isspace())

def get_name_from_srp_blob(clean_blob):
    blob_tokens = list(filter(bool, map(lambda x: x if len(x) > 2 else '', clean_blob.split(' '))))
    print(blob_tokens)
    c = Counter(blob_tokens)
    most_common = c.most_common(10)

    name = f"{most_common[0][0]}-{most_common[1][0]}"
    return name

pipeline = lambda x: get_name_from_srp_blob(blob_cleaner(get_srp_text(x)))

In [29]:
# topic_dict = {'Topic_' + str(i): [token for token, score in lda_bow.show_topic(i, topn=10)] for i in range(0, lda_bow.num_topics)}
# # print(topic_dict)
# for key in topic_dict:
#     joined = " ".join(topic_dict[key])
#     # print(joined)
#     name = pipeline(joined)
#     print(name)

topic_terms = "delivery area mile option partner traffic hub thanks city way"
name = pipeline(topic_terms)
print(name)

# topic_terms = "package address time customer apartment delivery number item support door"
# name = pipeline(topic_terms)
# print(name)

['mediumcom', 'sidewalktalk', 'thefutureoflastmiledeliveryhasarrived', 'deptswashingtonedu', 'sctlctr', 'newsevents', 'announcements', 'hub', 'wwwgeekwirecom', 'Science', 'wwwcapgeminicom', 'ReportDigitalLastMileDeliveryChallenge1', 'wwwridestercom', 'Food', 'Delivery', 'onfleetcom', 'blog', 'lastmiletechnologywhatitisandhowitisimpacti', 'wwwmckinseycom', 'industries', 'ourinsights', 'orderingintherapidevo', 'wwwbigcommercecom', 'blog', 'ecommerceshipping', 'wwworegonlivecom', 'business', '202009', 'amazonplans1000smalldel', 'booksgooglecom', 'books']
blog-mediumcom


Results obtained don't really make sense.

topic_terms = "delivery area mile option partner traffic hub thanks city way". Gives me ['mediumcom', 'sidewalktalk', 'thefutureoflastmiledeliveryhasarrived', 'deptswashingtonedu', 'sctlctr', 'newsevents', 'inthenews', 'futurelastmil', 'wwwcapgeminicom', 'ReportDigitalLastMileDeliveryChallenge1', 'wwwmckinseycom', 'travellogisticsandinfrastructure', 'ourinsights', 'wwwmckinseycom', 'industries', 'ourinsights', 'orderingintherapidevo', 'wwwbcgcom', 'publications', 'solvingthepackagedeliverysystemprobl', 'gigglefinancecom', 'whichfooddeliveryservicepaysthemost', 'wwwgovtmonitorcom', 'page', 'View', 'all', 'wwworegonlivecom', 'business', '202009', 'amazonplans1000smalldel']
wwwmckinseycom-ourinsights

Since there is no specific way or recommended way to label the topic terms obtained. We are going to do a little screening here and realise that the top few vocab being used do make some sense. Hence, in the following lines of code, we are going to assume and categorise the motive column into these 6 groups:

We will be using the topic breakdown from lda_bow, because they seem to have a more balanced categorisation.

*   Retaliation
*   Religion
*   Extortion
*   Fear
*   Political
*   Violence

Referenced: https://medcraveonline.com/FRCIJ/motivation-leading-to-radicalization-in-terrorists.html



## Evaluation of LDA

In [30]:
# Creating a dictionary to hold the catergories
motive_dict = {
    '0': "Retaliation",
    '1': "Religion",
    '2': "Extortion",
    '3': "Fear",
    '4': "Political",
    '5': "Violence"
}

In [31]:
# Extracting Topics from Copus
print(lda_bow.print_topics(num_topics=6, num_words=5))
print(lda_tfidf.print_topics(num_topics=6, num_words=5))

[(0, '0.016*"victim" + 0.015*"police" + 0.014*"government" + 0.012*"force" + 0.011*"member"'), (1, '0.013*"trend" + 0.013*"shiite" + 0.012*"community" + 0.011*"violence" + 0.011*"sunni"'), (2, '0.031*"islamic" + 0.023*"state" + 0.021*"levant" + 0.020*"iraq" + 0.020*"isil"'), (3, '0.025*"election" + 0.012*"islamic" + 0.010*"campaign" + 0.008*"liberation" + 0.008*"intimidate"'), (4, '0.027*"violence" + 0.021*"trend" + 0.016*"community" + 0.014*"sectarian" + 0.012*"police"'), (5, '0.016*"victim" + 0.015*"group" + 0.015*"government" + 0.013*"member" + 0.013*"retaliation"')]
[(0, '0.008*"extortion" + 0.008*"demand" + 0.007*"shiite" + 0.006*"islamic" + 0.006*"violence"'), (1, '0.012*"levant" + 0.011*"community" + 0.010*"shiite" + 0.010*"iraq" + 0.009*"sunni"'), (2, '0.006*"islamic" + 0.006*"election" + 0.006*"security" + 0.005*"state" + 0.005*"area"'), (3, '0.014*"election" + 0.008*"victim" + 0.008*"taliban" + 0.007*"government" + 0.007*"police"'), (4, '0.011*"bangladesh" + 0.010*"began" + 0

In [32]:
# https://stackoverflow.com/questions/61198009/classify-text-with-gensim-lda-model
count = 1
stop = 0
motiveCat=[]
for line in df['motive']: # where each line in the document is its own sentence for simplicity
    # print('\nSentence: ', line)
    # line = line.split()
    line_bow = dictionary.doc2bow(line)
    doc_lda = lda_bow[line_bow]
    # print(doc_lda)
    # print(max(doc_lda,key=lambda x:x[1])[0])
    # print('\nLine ' + str(count) + ' assigned to Topic ' + motive_dict[str(max(doc_lda)[0])] + ' with ' + str(round(max(doc_lda)[1]*100,2)) + ' probability!')
    count += 1
    motiveCat.append(motive_dict[str(max(doc_lda,key=lambda x:x[1])[0])])
    # if(str(max(doc_lda)[0])=='1'):
    #     print('\nSentence: ', line)
    #     print('\nLine ' + str(count) + ' assigned to Topic ' + motive_dict[str(max(doc_lda)[0])] + ' with ' + str(round(max(doc_lda)[1]*100,2)) + ' probability!')
    # stop+=1
    # if(stop==100):
    #     break
    
df['motiveCat1'] = motiveCat

In [33]:
df['motiveCat1'].value_counts()

Retaliation    5207
Political      4993
Fear           3744
Extortion      3261
Violence       3240
Religion       2821
Name: motiveCat1, dtype: int64

In [34]:
# https://stackoverflow.com/questions/61198009/classify-text-with-gensim-lda-model
count = 1
stop = 0
motiveCat=[]
for line in df['motive']: # where each line in the document is its own sentence for simplicity
    # print('\nSentence: ', line)
    # line = line.split()
    line_bow = dictionary.doc2bow(line)
    doc_lda = lda_tfidf[line_bow]
    # print(doc_lda)
    # print(max(doc_lda,key=lambda x:x[1])[0])
    # print('\nLine ' + str(count) + ' assigned to Topic ' + motive_dict[str(max(doc_lda)[0])] + ' with ' + str(round(max(doc_lda)[1]*100,2)) + ' probability!')
    count += 1
    motiveCat.append(motive_dict[str(max(doc_lda,key=lambda x:x[1])[0])])
    # if(str(max(doc_lda)[0])=='1'):
    #     print('\nSentence: ', line)
    #     print('\nLine ' + str(count) + ' assigned to Topic ' + motive_dict[str(max(doc_lda)[0])] + ' with ' + str(round(max(doc_lda)[1]*100,2)) + ' probability!')
    # stop+=1
    # if(stop==100):
    #     break
    
df['motiveCat2'] = motiveCat

In [35]:
df['motiveCat2'].value_counts()

Fear           4591
Religion       4222
Political      4016
Retaliation    3772
Extortion      3612
Violence       3053
Name: motiveCat2, dtype: int64

# Step 2: Dealing with Summary (Using the LDA models)
We will be using the previous lda_bow to categorise the motives for each row, we will using each summary with the motive categories.

## Data Pre-processing

In [36]:
# Converting all of the strings into lowercase
df['summary'] = df['summary'].str.lower()

# Removing punctuation
df['summary'] = df['summary'].str.replace('[{}]'.format(string.punctuation), '')

# Removing digits
df['summary'] = df['summary'].str.replace('\d+', '')

df['summary'].head

<input>:8: DeprecationWarning: invalid escape sequence \d
<input>:8: DeprecationWarning: invalid escape sequence \d
<input>:8: DeprecationWarning: invalid escape sequence \d
<ipython-input-36-9f7495e9d325>:8: DeprecationWarning: invalid escape sequence \d
  df['summary'] = df['summary'].str.replace('\d+', '')


<bound method NDFrame.head of 67506      the united nations special commission unscom ...
67511      two bombs exploded outside a private televisi...
67515      there was an explosion at the le meridien peg...
67518      the irish army disrupted an attack by disarmi...
67527      the loyalist volunteer force shot and killed ...
                                ...                        
201105     assailants opened fire on the vehicle of awas...
201118     assailants opened fire on boti lekam a former...
201127     assailants fired four rockets targeting a mil...
201175     assailants abducted  road transport administr...
201176     assailants abducted godlove abanda ndi in nja...
Name: summary, Length: 23266, dtype: object>

In [37]:
# for i, row in df.iterrows():
#     print(row['summary'])

In [38]:
# Removing stopwords
stop = stopwords.words('english')
newStopWords = ['unknown','however','specific','motive','sources','noted','reported','related','victim',\
                'attack','targeted','speculated','claimed','responsibility','incident','stated','scheduled',\
                'larger','part','suspected','carried','accused','may','january','february','march','april','june',\
                'july','august','september','october','november','december','people','believed','meant','considered']
stop.extend(newStopWords)

df['summary'] = df['summary'].apply(lambda x: [word for word in x.split() if word not in (stop)])
df['summary'].head

<bound method NDFrame.head of 67506     [united, nations, special, commission, unscom,...
67511     [two, bombs, exploded, outside, private, telev...
67515     [explosion, le, meridien, pegasus, hotel, geor...
67518     [irish, army, disrupted, disarming, car, bomb,...
67527     [loyalist, volunteer, force, shot, killed, cat...
                                ...                        
201105    [assailants, opened, fire, vehicle, awas, fade...
201118    [assailants, opened, fire, boti, lekam, former...
201127    [assailants, fired, four, rockets, targeting, ...
201175    [assailants, abducted, road, transport, admini...
201176    [assailants, abducted, godlove, abanda, ndi, n...
Name: summary, Length: 23266, dtype: object>

In [39]:
# Initialise the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize each word - Group together different forms of the same word (e.g Running == Run)
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]
df['summary'] = df['summary'].apply(lemmatize_text)
print(df['summary'].head)

<bound method NDFrame.head of 67506     [united, nation, special, commission, unscom, ...
67511     [two, bomb, exploded, outside, private, televi...
67515     [explosion, le, meridien, pegasus, hotel, geor...
67518     [irish, army, disrupted, disarming, car, bomb,...
67527     [loyalist, volunteer, force, shot, killed, cat...
                                ...                        
201105    [assailant, opened, fire, vehicle, awas, fadel...
201118    [assailant, opened, fire, boti, lekam, former,...
201127    [assailant, fired, four, rocket, targeting, mi...
201175    [assailant, abducted, road, transport, adminis...
201176    [assailant, abducted, godlove, abanda, ndi, nj...
Name: summary, Length: 23266, dtype: object>


## Bag of Words on the Dataset

In [40]:
# Create a dictionary from the df['motive'] containing the number of times a word appears in the training set.
dictionary = gensim.corpora.Dictionary(df['summary'])

# Print words that are in the top 10
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 attacked
1 baghdad
2 building
3 commission
4 damaged
5 firing
6 injured
7 iraq
8 nation
9 nobody
10 office


In [41]:
from sklearn.feature_extraction.text import CountVectorizer

# For each row, we will create another dictionary to document the number of words and the number of times that each words appear
vectorizer = CountVectorizer(stop_words='english', binary=True)

arr = []
for i in df['summary']:
    arr.append(' '.join(i))
df['summary'] = arr
bagofwords = vectorizer.fit_transform(df['summary'])

In [42]:
# df["summary"]= df["summary"].str.join(" ")
X = bagofwords
y = df['motiveCat1'].values

print(np.shape(X))
print(np.shape(y))
print(X[:1])
print(y[:10])

(23266, 38075)
(23266,)
  (0, 36002)	1
  (0, 24286)	1
  (0, 32827)	1
  (0, 8263)	1
  (0, 36040)	1
  (0, 25367)	1
  (0, 4038)	1
  (0, 16255)	1
  (0, 3558)	1
  (0, 26799)	1
  (0, 12274)	1
  (0, 29707)	1
  (0, 6419)	1
  (0, 9140)	1
  (0, 16024)	1
['Political' 'Fear' 'Fear' 'Fear' 'Political' 'Religion' 'Religion'
 'Violence' 'Violence' 'Violence']


In [43]:
# Splitting the dataset into Training set and Test set 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## KernelSVM

In [44]:
# Fitting Kernel SVM to the Training set 
from sklearn.svm import SVC 
model = SVC(kernel = 'rbf', random_state = 0) 
model.fit(X_train, y_train) 

SVC(random_state=0)

In [45]:
# Save Model
filename = '2_kernelSVM.sav'
pickle.dump(model, open(filename, 'wb'))

In [46]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import roc_auc_score

# Creating a definition for the eval part
def eval():
    # Predicting the Test set results
    y_pred = model.predict(X_test)

    # Making the Classification Report
    print(classification_report(y_test, y_pred))

    # Using the ROC AUC score matrix
    # roc_auc_score(y_test, y_pred, average=None)

In [47]:
eval()

              precision    recall  f1-score   support

   Extortion       0.67      0.51      0.58       649
        Fear       0.57      0.53      0.55       711
   Political       0.59      0.63      0.61      1004
    Religion       0.63      0.33      0.43       557
 Retaliation       0.41      0.68      0.51      1044
    Violence       0.60      0.34      0.43       689

    accuracy                           0.53      4654
   macro avg       0.58      0.50      0.52      4654
weighted avg       0.56      0.53      0.53      4654



## Logistic Regression (LR)

In [48]:
# Fitting Log Reg to the Training set 
model = LogisticRegression(solver="sag", max_iter=400)
model.fit(X_train, y_train)

LogisticRegression(max_iter=400, solver='sag')

In [49]:
# Save Model
filename = '2_logreg.sav'
pickle.dump(model, open(filename, 'wb'))

In [50]:
eval()

              precision    recall  f1-score   support

   Extortion       0.55      0.51      0.53       649
        Fear       0.54      0.54      0.54       711
   Political       0.57      0.62      0.60      1004
    Religion       0.49      0.40      0.44       557
 Retaliation       0.45      0.52      0.48      1044
    Violence       0.51      0.43      0.47       689

    accuracy                           0.52      4654
   macro avg       0.52      0.50      0.51      4654
weighted avg       0.52      0.52      0.52      4654



## Stochastic Gradient Descent (SGD)

In [51]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
model.fit(X_train, y_train)

SGDClassifier(max_iter=5)

In [52]:
# Save Model
filename = '3_sgd.sav'
pickle.dump(model, open(filename, 'wb'))

In [53]:
eval()

              precision    recall  f1-score   support

   Extortion       0.50      0.47      0.49       649
        Fear       0.52      0.40      0.45       711
   Political       0.56      0.46      0.51      1004
    Religion       0.40      0.38      0.39       557
 Retaliation       0.40      0.61      0.49      1044
    Violence       0.48      0.39      0.43       689

    accuracy                           0.47      4654
   macro avg       0.48      0.45      0.46      4654
weighted avg       0.48      0.47      0.47      4654



## Decision Tree (DT)

In [54]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
# Create Decision Tree classifer object
model = DecisionTreeClassifier()

# Train Decision Tree Classifer
model = model.fit(X_train,y_train)

In [55]:
# Save Model
filename = '4_dt.sav'
pickle.dump(model, open(filename, 'wb'))

In [56]:
eval()

              precision    recall  f1-score   support

   Extortion       0.50      0.53      0.52       649
        Fear       0.46      0.45      0.46       711
   Political       0.56      0.57      0.56      1004
    Religion       0.41      0.38      0.40       557
 Retaliation       0.41      0.45      0.43      1044
    Violence       0.44      0.38      0.41       689

    accuracy                           0.47      4654
   macro avg       0.46      0.46      0.46      4654
weighted avg       0.47      0.47      0.47      4654



In [57]:
# Visualizing Decision Trees
# !pip install six
# from six import StringIO  
# from IPython.display import Image  
# from sklearn.tree import export_graphviz
# import pydotplus

# feature_cols = ['Exortion', 'Fear', 'Political', 'Religion','Retaliation','Violence']
# dot_data = StringIO()
# export_graphviz(model, out_file=dot_data,  
#                 filled=True, rounded=True,
#                 special_characters=True, feature_names = feature_cols, class_names=['0','1'])
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# graph.write_png('diabetes.png')
# Image(graph.create_png())

## Random Forest (RF)

In [58]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=50)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50)

In [59]:
# Save Model
filename = '5_rf.sav'
pickle.dump(model, open(filename, 'wb'))

In [60]:
eval()

              precision    recall  f1-score   support

   Extortion       0.69      0.53      0.60       649
        Fear       0.54      0.55      0.54       711
   Political       0.56      0.67      0.61      1004
    Religion       0.65      0.38      0.48       557
 Retaliation       0.44      0.64      0.52      1044
    Violence       0.67      0.37      0.47       689

    accuracy                           0.54      4654
   macro avg       0.59      0.52      0.54      4654
weighted avg       0.57      0.54      0.54      4654



Interesting, that Random Forest also gives us a good accuracy here and in our previous question. Beating out the rest of the models that we have trained.

## k-Nearest Neighbors (kNN)

In [61]:
model = KNN(n_neighbors=7)
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [62]:
# Save Model
filename = '6_knn.sav'
pickle.dump(model, open(filename, 'wb'))

In [63]:
eval()

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:498: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  counts = np.zeros(a_view.shape[:-1], dtype=np.int)


              precision    recall  f1-score   support

   Extortion       0.48      0.50      0.49       649
        Fear       0.43      0.47      0.45       711
   Political       0.47      0.63      0.54      1004
    Religion       0.38      0.42      0.40       557
 Retaliation       0.46      0.34      0.39      1044
    Violence       0.48      0.31      0.38       689

    accuracy                           0.45      4654
   macro avg       0.45      0.45      0.44      4654
weighted avg       0.45      0.45      0.44      4654



## AdaBoost

In [64]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
model.fit(X_train, y_train)

AdaBoostClassifier()

In [65]:
# Save Model
filename = '7_adaBoost.sav'
pickle.dump(model, open(filename, 'wb'))

In [66]:
eval()

              precision    recall  f1-score   support

   Extortion       0.50      0.37      0.43       649
        Fear       0.41      0.43      0.42       711
   Political       0.55      0.39      0.45      1004
    Religion       0.36      0.07      0.12       557
 Retaliation       0.30      0.68      0.42      1044
    Violence       0.43      0.15      0.23       689

    accuracy                           0.38      4654
   macro avg       0.42      0.35      0.34      4654
weighted avg       0.42      0.38      0.36      4654



## Gradient Boosting

In [67]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

GradientBoostingClassifier()

In [68]:
# Save Model
filename = '8_gradientBoosting.sav'
pickle.dump(model, open(filename, 'wb'))

In [69]:
eval()

              precision    recall  f1-score   support

   Extortion       0.61      0.43      0.51       649
        Fear       0.52      0.45      0.49       711
   Political       0.55      0.52      0.54      1004
    Religion       0.62      0.21      0.31       557
 Retaliation       0.33      0.68      0.45      1044
    Violence       0.57      0.26      0.35       689

    accuracy                           0.46      4654
   macro avg       0.53      0.43      0.44      4654
weighted avg       0.52      0.46      0.45      4654



Gradient boosting and ADA Boosting are known to some of Ensemble Learning methods, and a boosting method. It is used to combine several weak "learners" into a stronger model. The concept of boosting is quite interesting because it tries to fit a new predictor into the errors that were made by models before.

Adaboost apparently gave us a lower accuracy 0.40 at predicting the motive categories.

GB obtaining an accuracy of 0.49 indicates that it was no better than running the previous models (without Boosting).

## K-Fold Cross Validation

In [70]:
# Applying k-Fold Cross Validation - not helpful right now
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = model, X = X_train, y = y_train, cv = 5) 

In [71]:
# Save Model
filename = '9_kfold.sav'
pickle.dump(accuracies, open(filename, 'wb'))

In [72]:
print(accuracies.mean())
print(accuracies.std())

0.46856866483279286
0.006292202097072469


Using the K-Fold Cross Validation should improve the model in general, however in this case, we would attribute it to a case of a little over-fitting. The other models that were executed, had also an average accuracy of 0.49. Hence we can conclude that executing K-Fold Cross Validation does not help us in this case.

# Step 3: Evaluation
Summing everything up, we have used LDA to determine the motive categories and obtained 6 of them that we think would be most suitable to represent the topic labels.

*   Retaliation
*   Religion
*   Extortion
*   Fear
*   Political
*   Violence

The above are the reasons and the main goals of the terrorists for initiating such attacks.

Taking a look at the perfomance metrices for each model. We have learnt from doing the previous question "What makes a successful terrorist attack?" that the accuracy cannot entirely be depended on:

*   Confusion Matrix - Great for visualisation (Based on TP,TN,FN,FP)
*   Accuracy - Not a very good indicator. Most commonly used, the number of correct predictions made as a ratio of all predictions
*   Precision - TP/(TP+FP). This is the measure of precision.
*   Recall - TP/(TP+FN). This is the measure of precision.
*   F1-score - This is the measure of precision and recall. Taking FP and FN into account.
*   ROC-AUC score - AUC represents an average indicator of performance across all the possible classification thresholds.

Reference: <br>
Performance Metrics: https://machinelearningmastery.com/metrics-evaluate-machine-learning-algorithms-python/



## Conclusion
We also understand that through this process, we have significantly cut down on alot of data points. Going to columns, and after preprocessing the data for NLP, the once 200k data points in it's glory is reduced to around 20k of them. However, it is still significant enough for us to draw analysis.

Now to finally answer the question: "What do terrorists really want?". We can see that terrorists usually attack civilians to maximise their political gains (as we can see the political category having the highest number of datapoints = 966). Perhaps it's to make a statement and coming from a place of wanting to be heard. Although the models created were not exactly extremely accurate when determining the categories, being able to use LDA to determine the most common frequencies of words really helps us to understand the motives better, and hence group them under 6 topic labels. 

One of the ways that we thought could help in countering such terrorist attacks is to dimininish their benefits from the attack. Meaning that since we can know of their aims beforehand, we would be able to strategise against their goals and void the attack, hopefully stopping it in the early planning stages.

Originally, we wanted to use the models to try and help predict certain keywords that appear, like their country, situation, some background info that was obtained before the attacks was executed. From there, being able to predict what the the terrorists want before them stating it to us.